<a href="https://colab.research.google.com/github/aycaecemgul/Torch/blob/main/torch_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ⚡ **Pytorch 101** ⚡




# Tensors

Initializing a tensor

In [ ]:
#directly from data
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)

In [ ]:
#from another tensor
x_ones = torch.ones_like(x_data) # retains the properties of x_data

x_rand = torch.rand_like(x_data, dtype=torch.float) # overrides the datatype of x_data

In [ ]:
#attributes of tensor
tensor = torch.rand(3,4)

print(f"Shape of tensor: {tensor.shape}")
print(f"Datatype of tensor: {tensor.dtype}")
print(f"Device tensor is stored on: {tensor.device}")

In [ ]:
#With random or constant values
shape = (2,3,)
rand_tensor = torch.rand(shape)
ones_tensor = torch.ones(shape)
zeros_tensor = torch.zeros(shape)

# Manipulating Tensor Shapes

The unsqueeze() method adds a dimension of extent 1. unsqueeze(0) adds it as a new zeroth dimension - now you have a batch of one!

In [ ]:
a = torch.rand(3, 226, 226)
b = a.unsqueeze(0)

#out:
# torch.Size([3, 226, 226])
# torch.Size([1, 3, 226, 226])

Continuing the example above, let’s say the model’s output is a 20-element vector for each input. You would then expect the output to have shape (N, 20), where N is the number of instances in the input batch. That means that for our single-input batch, we’ll get an output of shape (1, 20).

What if you want to do some non-batched computation with that output - something that’s just expecting a 20-element vector?

In [ ]:
c = torch.rand(2, 2)
print(c.shape)

d = c.squeeze(0)
print(d.shape)

n-dimentional tensor to 1 dimention tensor

In [ ]:
output3d = torch.rand(6, 20, 20)
print(output3d.shape)

input1d = output3d.reshape(6 * 20 * 20)
print(input1d.shape)


Numpy to torch

In [ ]:
import numpy as np

numpy_array = np.ones((2, 3))
print(numpy_array)

pytorch_tensor = torch.from_numpy(numpy_array)
print(pytorch_tensor)

Torch to numpy

In [ ]:
pytorch_rand = torch.rand(2, 3)
print(pytorch_rand)

numpy_rand = pytorch_rand.numpy()
print(numpy_rand)

# Handling Data & Datasets

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

PyTorch has two primitives to work with data: **torch.utils.data.DataLoader** and **torch.utils.data.Dataset**. Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset.
The **torchvision.datasets **module contains Dataset objects for many real-world vision data like CIFAR, COCO

**root** is the path where the train/test data is stored,
**train** specifies training or test dataset,
**download=True** downloads the data from the internet if it’s not available at root.
**transform** and **target_transform **specify the feature and label transformations

In [ ]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

**DataLoader** wraps an iterable over our dataset, and supports a**utomatic batching, sampling, shuffling and multiprocess data loading**. Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [ ]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Creating a custom dataset for your files

A custom Dataset class must implement three functions: __init__, __len__, and __getitem__.

images are stored in a directory img_dir, and their labels are stored separately in a CSV file annotations_file

In [ ]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, img_dir, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(annotations_file)
        self.img_dir = img_dir
        self.transform = transform #transform methods for images
        self.target_transform = target_transform #transform methods for labels

    #returns the number of samples 
    def __len__(self):
        return len(self.img_labels)
    
    #returns image on a given id
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

 Each iteration below returns a batch of **train_features** and **train_labels** **(containing batch_size=64 features and labels respectively)**. Because we specified **shuffle=True**, after we iterate over all batches the data is shuffled

In [ ]:
train_features, train_labels = next(iter(train_dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label}")

In [ ]:
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

# Creating Models


To define a neural network in PyTorch, we create a class that inherits from** nn.Module** We define the layers of the network in the __init__ function and specify how data will pass through the network in the **forward** function.

In [ ]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    #connect layers using forward
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


# Training & Optimizing the Model Parameters

The **gradient** is a vector which gives us the direction in which loss function has the steepest ascent. Exactly opposite to the direction of the gradient is which we have to move in.(moving along which direction brings about the steepest decline in the value of the loss function) We perform **descent** along the direction of the gradient, hence, it's called **Gradient Descent**. Now, once we have the direction we want to move in, we must decide the size of the step we must take. The the size of this step is called the **learning rate**. We must chose it carefully to ensure we can get down to the minima. **Once we have our gradient and the learning rate, we take a step, and recompute the gradient at whatever position we end up at, and repeat the process.**

**step()** makes the optimizer iterate over all parameters (tensors) it is supposed to update and use their internally stored grad to update their values.


When you create a tensor, if you set its attribute .requires_grad as True, the package tracks all operations on it. This happens on subsequent backward passes. The gradient for this tensor will be accumulated into .grad attribute. The accumulation (or sum) of all the gradients is calculated when .backward() is called on the loss tensor.

In [ ]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y) # Compute prediction error

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

While the direction of the gradient tells us which direction has the steepest ascent, it's magnitude tells us how steep the steepest ascent/descent is. So, at the minima, where the contour is almost flat, you would expect the gradient to be almost zero. In fact, it's precisely zero for the point of minima.

We subtract the gradient of the loss function with respect to the weights multiplied by alpha, the learning rate. The gradient is a vector which gives us the direction in which loss function has the steepest ascent. The direction of steepest descent is the direction exactly opposite to the gradient, and that is why we are subtracting the gradient vector from the weights vector. Before subtracting we multiply the gradient vector by the learning rate. 

We can afford a large learning rate. But later on, we want to slow down as we approach a minima. An approach that implements this strategy is called **Simulated annealing, or decaying learning rate.** In this, the learning rate is decayed every fixed number of iterations.

Gradient descent is driven by the gradient, which will be zero at the base of any minima. Local minimum are called so since the value of the loss function is minimum at that point in a local region. Whereas, a global minima is called so since the value of the loss function is minimum there, globally across the entire domain the loss function.

A way to help GD escape saddle point is to use what is called Stochastic Gradient Descent.  instead of taking a step by computing the gradient of the loss function creating by summing all the loss functions, we take a step by computing the gradient of the loss of only one randomly sampled example. from a theoretical standpoint, stochastic gradient descent might give us the best results, it's not a very viable option. So, what we do is a balancing act. Instead of using the entire dataset, or just a single example to construct our loss function, we use a fixed number of examples say, 16, 32 or 128 to form what is called a mini-batch. The word is used in contrast with processing all the examples at once, which is generally called Batch Gradient Descent. 


# How to use an optimizer


To use torch.optim you have to construct an optimizer object, that will hold the current state and will update the parameters based on the computed gradients.
To construct an Optimizer you have to give it an iterable containing the parameters (all should be Variable s) to optimize. Then, you can specify optimizer-specific options such as the learning rate, weight decay, etc.

In [ ]:
#example
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam([var1, var2], lr=0.0001)

Optimizer s also support specifying per-parameter options. To do this, instead of passing an iterable of Variable s, pass in an iterable of dict s. Each of them will define a separate parameter group, and should contain a params key, containing a list of parameters belonging to it. Other keys should match the keyword arguments accepted by the optimizers, and will be used as optimization options for this group.

This means that model.base’s parameters will use the default learning rate of 1e-2, model.classifier’s parameters will use a learning rate of 1e-3, and a momentum of 0.9 will be used for all parameters.

In [ ]:
#example
optim.SGD([
                {'params': model.base.parameters()},
                {'params': model.classifier.parameters(), 'lr': 1e-3}
            ], lr=1e-2, momentum=0.9)

All optimizers implement a step() method, that updates the parameters. It can be used in two ways:

optimizer.step()
This is a simplified version supported by most optimizers. The function can be called once the gradients are computed using e.g. backward().

When you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly. Otherwise, the gradient would be a combination of the old gradient, which you have already used to update your model parameters, and the newly-computed gradient. It would therefore point in some other direction than the intended direction towards the minimum

The accumulation (or sum) of all the gradients is calculated when .backward() is called on the loss tensor. There are cases where it may be necessary to zero-out the gradients of a tensor. For example: when you start your training loop, you should zero out the gradients so that you can perform this tracking correctly.

In [ ]:
for input, target in dataset:
    optimizer.zero_grad()
    # Compute prediction error
    pred = model(input)
    loss = loss_fn(pred, target)
    loss.backward()
    optimizer.step()

You can also use model.zero_grad(). This is the same as using optimizer.zero_grad() as long as all your model parameters are in that optimizer.

# Testing the model

In [ ]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

# Saving the model

In [ ]:
torch.save(model.state_dict(), "model.pth")

# Loading the model

The process for loading a model includes re-creating the model structure and loading the state dictionary into it.



In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

# Evaluation

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

# Transforms

transforms.ToTensor() converts images loaded by Pillow into PyTorch tensors.
transforms.Normalize() adjusts the values of the tensor so that their average is zero and their standard deviation is 0.5. Most activation functions have their strongest gradients around x = 0, so centering our data there can speed learning.

# Moving to GPU

One of the major advantages of PyTorch is its robust acceleration on CUDA-compatible Nvidia GPUs. (“CUDA” stands for Compute Unified Device Architecture, which is Nvidia’s platform for parallel computing.) So far, everything we’ve done has been on CPU.

In [ ]:
if torch.cuda.is_available():
    print('We have a GPU!')
else:
    print('Sorry, CPU only.')

Once we’ve determined that one or more GPUs is available, we need to put our data someplace where the GPU can see it. Your CPU does computation on data in your computer’s RAM. Your GPU has dedicated memory attached to it. Whenever you want to perform a computation on a device, you must move all the data needed for that computation to memory accessible by that device. By default, new tensors are created on the CPU, so we have to specify when we want to create our tensor on the GPU with the optional **device** argument. in order to do computation involving two or more tensors, all of the tensors must be on the same device. 